In [1]:
print("Welcome to BoxWorks Master Class")

Welcome to BoxWorks Master Class


# Deep dive into Box AI API

Prerequisites: 
1. Enable Box AI in Admin Console
2. Create a Box Platform app
3. Enable Manage AI scope in app's configuration tab
4. Generate a developer token

## Install Box Python SDK

In [7]:
pip install box-sdk-gen

Note: you may need to restart the kernel to use updated packages.


## Authenticate with a developer token and the get current user ID

In [68]:
from box_sdk_gen import BoxClient, BoxDeveloperTokenAuth

auth = BoxDeveloperTokenAuth(token="TOKEN")
client = BoxClient(auth=auth)

me = client.users.get_user_me()
print(f"My user ID is {me.id}")

My user ID is 21278765957


## Ask AI to summarise the document

In [12]:
from box_sdk_gen import CreateAiAskMode, AiItemAsk

In [64]:
client.ai.create_ai_ask(
    CreateAiAskMode.SINGLE_ITEM_QA,
    "summarize this document",
    [
        AiItemAsk(
            id="1925154254074",
            type="file"
        )
    ]
)

<class 'box_sdk_gen.schemas.ai_response_full.AiResponseFull'> {'answer': 'This document presents a novel process-centric taxonomy for organizing, classifying, and synthesizing tactile robot manipulation skills, addressing the challenge of scalable and reliable skill deployment in robotics. The authors introduce a formal framework that maps expert-defined process specifications to tactile skills through a hierarchical taxonomy, enabling efficient selection and learning of manipulation policies tailored to specific tasks such as insertion, cutting, and assembly. They implement this approach using the Graph-Guided Twist-Wrench Policy (GGTWreP) framework on a 7-DoF Franka Emika robot with a two-finger gripper, demonstrating robust performance across 28 industrially relevant skills with near 100% success rates even under disturbances. The framework integrates control, learning, and tactile feedback, allowing parameter adaptation and transferability between related skills. Compared to deep r

## Display the AI response in a readable format

In [86]:
def display_ai_response(response):
    """
    Display the AI response in a readable format
    """
    # Extract the data from the response object
    answer = response.answer if hasattr(response, 'answer') else "No answer available"
    created_at = response.created_at if hasattr(response, 'created_at') else "Unknown"
    completion_reason = response.completion_reason if hasattr(response, 'completion_reason') else "Unknown"
    
    # Print in a readable format
    print("=== AI RESPONSE ===")
    print(answer)
    print(f"\nGenerated at: {created_at}")
    print(f"Completion status: {completion_reason}")
    
    # Get model info if available
    if hasattr(response, 'ai_agent_info') and hasattr(response.ai_agent_info, 'models'):
        if response.ai_agent_info.models:
            print(f"AI model used: {response.ai_agent_info.models[0].name}")

summary = client.ai.create_ai_ask(
    CreateAiAskMode.SINGLE_ITEM_QA,
    "Summarize this document",
    [
        AiItemAsk(
            id="1925154254074",
            type="file"
        )
    ]
)
display_ai_response(summary)

=== AI RESPONSE ===
This document presents a novel process-centric taxonomy for organizing, classifying, and synthesizing tactile robot manipulation skills to improve the deployment and learning of autonomous robotic manipulation in industrial settings. The authors introduce a formal framework that links expert-defined process specifications to tactile skill implementations via a hierarchical taxonomy, enabling scalable and efficient skill selection and learning. They validate their approach experimentally on 28 diverse real-world manipulation tasks using the Graph-Guided Twist-Wrench Policy (GGTWreP) framework with a standard 7-DoF manipulator, achieving near 100% success rates and robust performance under disturbances. The work emphasizes integrating control, learning, and process constraints, reducing reliance on robotics expertise by leveraging established human vocational curricula as a knowledge base. Compared to deep reinforcement learning methods, their approach is more energy-

## Query a different content type: image

We'll create a simple function that determines if given file is a passport page.

In [57]:
def is_passport(file_id):
    response = client.ai.create_ai_ask(
        CreateAiAskMode.SINGLE_ITEM_QA,
        "Is this document a passport page? Return only YES or NO",
        [
            AiItemAsk(
                id=file_id,
                type="file"
            )
        ]
    )
    
    return response

display_ai_response(is_passport("1947306147694"))
display_ai_response(is_passport("1947326375742"))

=== AI RESPONSE ===
YES

Generated at: 2025-08-12 04:20:07.009000-07:00
Completion status: done
AI model used: azure__openai__gpt_4_1_mini
=== AI RESPONSE ===
NO

Generated at: 2025-08-12 04:20:09.271000-07:00
Completion status: done
AI model used: azure__openai__gpt_4_1_mini


## Translate document with Box AI

In [54]:
Need to rewrite to ask endpoint
# from box_sdk_gen import CreateAiTextGenItems, CreateAiTextGenItemsTypeField

# def translate_document(file_id, language):
#     response = client.ai.create_ai_text_gen(
#         f"""Translate this document to {language}""",
#         [
#             CreateAiTextGenItems(
#                 id=file_id,
#                 type=CreateAiTextGenItemsTypeField.FILE,
#             )
#         ],
#     )
#     return response

# display_ai_response(translate_document("1925154254074", "Polish"))

=== AI RESPONSE ===
Please provide the document you want me to translate into Polish.

Generated at: 2025-08-12 03:52:12.017000-07:00
Completion status: done
AI model used: azure__openai__gpt_4_1_mini


## Get default AI API configuration

In [80]:
from box_sdk_gen import GetAiAgentDefaultConfigMode

client.ai.get_ai_agent_default_config(GetAiAgentDefaultConfigMode.ASK, language="en-US")

<class 'box_sdk_gen.schemas.ai_agent_ask.AiAgentAsk'> {'type': 'ai_agent_ask', 'long_text': {'model': 'azure__openai__gpt_4_1_mini', 'num_tokens_for_completion': 6000, 'llm_endpoint_params': {'type': 'openai_params', 'temperature': 0, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 1.5, 'stop': '<|im_end|>'}, 'prompt_template': 'Current date: {current_date}\n\nI will ask you for help and provide subsections of one document delimited by five backticks (`````) at the beginning and at the end.\nIf I make a reference to "this", I am referring to the document I provided between the five backticks. I may ask you a question where the answer is contained within the document.  In that case, do your best to answer using only the document, but if you cannot, feel free to mention that you couldn\'t find an answer in the document, but you have some answer from your general knowledge.\nI may ask you to perform some kind of computation or symbol manipulation such as filtering a list, counting

## Format config output

In [85]:
def display_ai_config(response):
    """Display the AI config in a readable format"""
    
    def safe_getattr(obj, attr, default=None):
        """Safely get attribute with default value"""
        return getattr(obj, attr, default) if hasattr(obj, attr) else default
    
    def print_field(label, value, truncate_at=100):
        """Print field with optional truncation"""
        if value is None:
            return
        value_str = str(value)
        if len(value_str) > truncate_at:
            print(f"{label}: {value_str[:truncate_at]}...")
        else:
            print(f"{label}: {value_str}")
    
    # Header
    print("=== AI CONFIG ===")
    print(f"\nAgent type: {safe_getattr(response, 'type', 'No agent type available')}")
    
    # Model types to check
    model_types = [
        'long_text', 'basic_text', 'spreadsheet', 'long_text_multi',
        'basic_text_multi', 'basic_image', 'basic_image_multi'
    ]
    
    # LLM parameter mappings
    llm_params_map = {
        'type': 'Type',
        'temperature': 'Temperature',
        'top_p': 'Top-p',
        'frequency_penalty': 'Frequency penalty',
        'presence_penalty': 'Presence penalty',
        'stop': 'Stop sequence'
    }
    
    for model_type in model_types:
        model_config = safe_getattr(response, model_type)
        if not model_config:
            continue
        
        print(f"\n--- {model_type.upper().replace('_', ' ')} ---")
        
        # Basic fields
        print_field("Model", safe_getattr(model_config, 'model'))
        print_field("Max tokens", safe_getattr(model_config, 'num_tokens_for_completion'))
        
        # LLM parameters
        llm_params = safe_getattr(model_config, 'llm_endpoint_params')
        if llm_params:
            print("LLM Parameters:")
            for attr, label in llm_params_map.items():
                value = safe_getattr(llm_params, attr)
                if value is not None:
                    print(f"  {label}: {value}")
        
        # System message and prompt template
        print_field("System message", safe_getattr(model_config, 'system_message'))
        print_field("Prompt template", safe_getattr(model_config, 'prompt_template'))
        
        # Embeddings configuration
        embeddings = safe_getattr(model_config, 'embeddings')
        if embeddings:
            print("Embeddings:")
            print_field("  Model", safe_getattr(embeddings, 'model'))
            
            strategy = safe_getattr(embeddings, 'strategy')
            if strategy:
                print_field("  Strategy", safe_getattr(strategy, 'id'))
                print_field("  Tokens per chunk", safe_getattr(strategy, 'num_tokens_per_chunk'))
    
    print("\n=== END AI CONFIG ===")

display_ai_config(client.ai.get_ai_agent_default_config(GetAiAgentDefaultConfigMode.ASK, language="en-US"))

BoxSDKError: 
Timestamp: 2025-08-12 15:31:48.811358
Underlying error: None
Message: Developer token has expired. Please provide a new one.

## Get default config for text gen endpoint

In [84]:
client.ai.get_ai_agent_default_config(GetAiAgentDefaultConfigMode.TEXT_GEN, language="en-US")

# display_ai_config(client.ai.get_ai_agent_default_config(GetAiAgentDefaultConfigMode.TEXT_GEN, language="en-US"))

<class 'box_sdk_gen.schemas.ai_agent_text_gen.AiAgentTextGen'> {'type': 'ai_agent_text_gen', 'basic_gen': {'model': 'azure__openai__gpt_4_1_mini', 'num_tokens_for_completion': 12000, 'llm_endpoint_params': {'type': 'openai_params', 'temperature': 0.1, 'top_p': 1, 'frequency_penalty': 0.75, 'presence_penalty': 0.75, 'stop': '<|im_end|>'}, 'system_message': '\nIf you need to know today\'s date to respond, it is {current_date}.\nThe user is working in a collaborative document creation editor called Box Notes.\nIf the user simply asks to "improve" the text, then simplify the language and remove jargon, unless the user specifies otherwise.\nTry your best to format the document nicely using Markdown.  When you use headers, make them prominent. Be comprehensive in your answer. When asked to create or modify lists, similar formatting requests, provide the list of content without adding titles, headers, introductory text, or concluding explanations unless specifically requested.\n', 'prompt_tem

## Overwrite default config



4. Generate summaries from files in one folder and generate an index Note. > what's the limit of number of files
5. Query Box Hubs
6. Query a spreadsheet
7. Translate a document with multiple documents
8. Free form extruct warious formats
9. Extract structured + apply metadata template
10. Use extract enhanced agent
12. Show the JSON configurator
13. Owerwrite the default model
14. Talk about max tokens
15. Define system prompt
16. Show examples of temperature modification
17. Create Studio AI agent
